In [37]:
############################################################
"""
    Importing initial Notebooks and Libraries
"""
############################################################

# Jupyter notebook imports
%run twitter_sentiment.ipynb

# Library imports
import os
import pandas as pd
from datetime import date, timedelta
import alpaca_trade_api as tradeapi
from talib import RSI, OBV, ATR, STDDEV

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

Twitter Authentication Verified


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Kris/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [38]:
############################################################
"""
    API Authentications for Alpaca
"""
############################################################
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [39]:
def get_indicators(symbol, security):   

    indicator_scores = []
    # column_names = ['symbol']
    indicator_scores = pd.DataFrame() #columns = column_names)
    
    # Set timeframe to 15 Minutes
    timeframe = "15Min"
    
    # Set end date to now for latest data
    end_date = pd.Timestamp.now(tz="America/New_York").isoformat()
    
    # set limit to past 100 15-minute intervals (equal to two trading days of 6.5 hours)
    limit = 100
        
    #fetch ticker data 
    ticker_data = api.get_barset(
        ticker,
        timeframe,
        limit,
        end=end_date
        )[ticker]._raw
    ticker_data_df = pd.DataFrame(data=ticker_data)
    
    ############################################################
    """ 
        Volatility - Average True Range
        Finds volatile stocks using ATR.
        Looks for stocks that typically move more than 5% per day, based on a 50 time period average, 
        
        Relative strength suggests continued outperformance while relative weakness suggests continued underperformance.
        High RSI (usually above 70) may indicate a stock is overbought, therefore it is a sell signal. 
        Low RSI (usually below 30) indicates stock is oversold, which means a buy signal. 
        
        On-balance volume (OBV) is a technical trading momentum indicator that 
        uses volume flow to predict changes in stock price.
        
        When both price and OBV are making higher peaks and higher troughs, 
        the upward trend is likely to continue.
    """
    ############################################################
    
    #fetch Average True Range(ATR) volatility indicator
    #average_true_range = pd.DataFrame(ATR(ticker_data_df['h'], ticker_data_df['l'], ticker_data_df['c'], timeperiod=14))

    #fetch Standard Deviation volatility indicator
    #std_dev = pd.DataFrame(STDDEV(ticker_data_df['c'], timeperiod=5, nbdev=1))
    std_dev = pd.DataFrame(ticker_data_df['c']).std()
    std_dev = std_dev.mean()
                           
    #Fetch RSI score
    rsi_score = pd.DataFrame(RSI(ticker_data_df['c'], timeperiod=14))
    #rsi_score = rsi_score.mean()
    rsi_score = rsi_score.loc[(len(rsi_score)-1), 0]
     
    #Fetch On Balance Volume(OBV)
    obv_score = pd.DataFrame(OBV(ticker_data_df['c'], ticker_data_df['v']))
    
    if (rsi_score <= 45) and (std_dev > 0.001):
    
        # if all conditions met, appends scores to dataframe
        # Set the symbol row and column

        indicator_scores.loc[0, 'symbol'] = symbol
        indicator_scores.loc[0, 'security'] = security
        indicator_scores.loc[0, 'std_dev'] = std_dev                    
        #indicator_scores.loc[0, 'avg_true_range'] = average_true_range.loc[(len(average_true_range)-1), 0]
        indicator_scores.loc[0, 'on_balance_volume'] = obv_score.loc[(len(obv_score)-1), 0]
        indicator_scores.loc[0, 'relative_strength_index'] = rsi_score
        
    else:
        pass
    
    return indicator_scores

############################################################
"""
    Function that fetches the Twitter Sentiment score from twitter_sentiment.ipynb
"""
############################################################

def fetch_twitter_sentiment(ticker, search_word):
    score = compound_twitter_sentiment(ticker, search_word)
    return score


In [77]:
def stock_screener():

    ############################################################
    """
        Scrapes Wikipedia and pulls stock data for every ticker symbol on the S&P500
    """
    ############################################################
    
    # scrapes the wikipedia page relating to the S&P 500 and returns a list of DataFrame objects
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    
    # Since we are only interested in the current list of stocks in the S&P 500, we only need the DataFrame object at index 0
    sp500_list_df = table[0]
    sp500_tickers = sp500_list_df[["Symbol", "Security"]]
    
    ############################################################
    """
        Stock Screener for Day Trading
        Start Main Function
    """
    ############################################################
    
    # Initialize the dataframe and set the columns
    indicators_df = []
    column_names = ['symbol', 'security', 'std_dev', 'on_balance_volume', 'relative_strength_index']
    indicators_df = pd.DataFrame(columns = column_names)
    
    # for loop iterates through all tickers in S&P 500
    for tickers in range(len(sp500_tickers["Symbol"])):
        
        # set the current ticker symbol and company security name
        company = sp500_tickers.loc[[tickers][0]]
        ticker = company.Symbol
        security = company.Security
        
        # Set timeframe to 1 Day to capture daily Volume total
        timeframe = "1D"
        
        # Set the limit of bars to just the previous time period
        limit = 1
    
        # Set end date to now for latest data (if run before market open to fetch yesterdays close)
        end_date = pd.Timestamp.now(tz="America/New_York").isoformat()
        
        ############################################################
        """    
        
        *** Notes on how to iterate over a barset object from Alpaca ***
        
        # Set timeframe to 15 Minutes
        timeframe = "15Min"
        #fetch ticker data 
        ticker_data = api.get_barset(
            ticker,
            timeframe,
            limit,
            end=end_date
            )
            
        # ticker_data inherits dict and bars inherits list. You can iterate over them accordingly: 
        for symbols in ticker_data:
            bars = ticker_data[symbols]
            for bar in bars:
                sp500_ticker_data.loc[symbols, 'time'] = bar.t
                sp500_ticker_data.loc[symbols, 'open'] = bar.o
                sp500_ticker_data.loc[symbols, 'low'] = bar.l
                sp500_ticker_data.loc[symbols, 'high'] = bar.h
                sp500_ticker_data.loc[symbols, 'close'] = bar.c
                sp500_ticker_data.loc[symbols, 'volume'] = bar.v
                
        """
        ############################################################ 
        
        # Initial pass of ticker symbols
        # fetch last daily barset object for ticker and put raw data into dataframe
        bar_set = api.get_barset(
            ticker,
            timeframe,
            limit,
            end=end_date
            )[ticker]._raw
        initial_check = pd.DataFrame(data=bar_set)
        
        ############################################################
        """
        # Since Day traders generally look for stocks that have at least 1 million shares traded daily, 
        # this checks to see if the daily volume for current ticker >= 1Million.
        # This also checks the last closing price of a ticker to see if it is below $1 or above $100.
        # If ticker volume < 1M or it's last closing price was above $100, 
        # we stop this iteration and continue to the next iteration of the loop.
        """
        ############################################################
               
        if (initial_check.loc[0, 'v'] <= 1000000) or (initial_check.loc[0, 'c'] > 100) or (initial_check.loc[0, 'c'] < 1):
            continue
        else:
            # If the ticker passes initial checks, it continues to calculating the additional indicators
            
            indicators_df = indicators_df.append(get_indicators(ticker, security), ignore_index=True)
            
    ############################################################
    """ 
        Twitter and Reddit Compound Sentiment
    """
    ############################################################
        
    for stocks in range(len(indicators_df["symbol"])):
        
        ticker = indicators_df.loc[stocks, "symbol"]
        security = indicators_df.loc[stocks, "security"]
        
        #fetches the Compound Sentiment score for the last 15 minutes of Tweets on Twitter
        twitter_comp_sentiment_score = fetch_twitter_sentiment(ticker, security)
        indicators_df.loc[stocks, 'twitter_sentiment'] = twitter_comp_sentiment_score
            
        #fetch reddit sentiment
        #reddit_sentiment = fetch_reddit_sentiment(ticker, security)
        #indicators_df.loc[x, 'reddit_sentiment'] = reddit_sentiment
    
    
        
    ############################################################
    """ 
    
        We want a stock that scores low on the RSI (oversold) and high on the Social Sentiment (people are talking good about it, may signal an uptrend).
        This calulates the score for RSI - (Total Social Sentiment * 100) and returns a sorted dataframe with Top 5 stock tickers (in this case, the 5 lowest scores in ascending order).
        We multiply the Total Social Sentiment by 100 to scale it for RSI.
        
    """
    ############################################################    
        
    # adds up sentiment indicator column scores and calculates the total_indi_score
    for symbol in range(len(indicators_df["symbol"])):
        indicators_df.loc[symbol, 'total_screener_score'] = (indicators_df.loc[symbol, 'relative_strength_index'] - (indicators_df.loc[symbol, 'twitter_sentiment'] * 100)) # + indicators_df.loc[rows, 'reddit_sentiment'])
    
    # sets the index to the ticker symbol and sorts indidcator_df by score
    indicators_df.set_index('symbol', inplace=True)
    indicators_df.sort_values(by='total_screener_score', inplace=True)
    
    # saves top 5 tickers into dataframe
    top5_stocks_df = indicators_df[:5]
    top5 = pd.DataFrame(data = top5_stocks_df.index)

    
    # return top 5 stock recommendations
    return top5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [76]:

top5 = stock_screener()
top5

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://da

TweepError: Twitter error response: status code = 503